In [0]:
import requests
from pyspark.sql.functions import lit

In [0]:
def extract_data(date, base="BRL"):

    url = f"https://api.apilayer.com/exchangerates_data/{date}&base={base}"

    headers= {
    "apikey": "26fIuqF2C5C9BA52xsBxEF8YaAiXnddd"
    }

    parameters = {
        "base":base,
        "symbols": "USD,GBP,EUR"
        }
    

    response = requests.request("GET", url, headers=headers, params=parameters)

    if response.status_code != 200:
        raise Exception(f'Request failed with status {response.status_code}')
    return response.json()


In [0]:
def data_to_df (data_json): 
    data_tuple = [(moeda, float (taxa)) for moeda, taxa in data_json["rates"].items()]
    
    return data_tuple

In [0]:
def save_parquet(extracted_conversions):
    year, month, day = extracted_conversions['date'].split('-')
    path = f"dbfs:/databricks-results/bronze/{year}/{month}/{day}"
    response = data_to_df(extracted_conversions)
    df_conversions = spark.createDataFrame(response, schema=["moeda", "taxa"])
    df_conversions = df_conversions.withColumn("date", lit(f"{year}-{month}-{day}"))

    df_conversions.write.format("parquet").mode("overwrite").save(path)

    print(f"Data saved to {path}")


In [0]:
exchange_rates = extract_data("2025-07-20")
save_parquet(exchange_rates)

In [0]:
display(dbutils.fs.ls("dbfs:/databricks-results/bronze/2025/07/20"))